# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [12]:
import pandas as pd 
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [13]:

df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
# Step 2: Filter customers
low_claim_responders = df[
    (df['total_claim_amount'] < 1000) &
    (df['response'].str.lower() == 'yes')
].copy()

# Step 3: View the result
print(low_claim_responders.head())

    unnamed:_0 customer       state  customer_lifetime_value response  \
3            3  XL78013      Oregon             22332.439460      Yes   
8            8  FM55990  California              5989.773931      Yes   
15          15  CW49887  California              4626.801093      Yes   
19          19  NJ54277  California              3746.751625      Yes   
27          27  MQ68407      Oregon              4376.363592      Yes   

    coverage education effective_to_date employmentstatus gender  ...  \
3   Extended   College           1/11/11         Employed      M  ...   
8    Premium   College           1/19/11         Employed      M  ...   
15     Basic    Master           1/16/11         Employed      F  ...   
19  Extended   College           2/26/11         Employed      F  ...   
27   Premium  Bachelor           2/28/11         Employed      F  ...   

    number_of_open_complaints number_of_policies     policy_type  \
3                         0.0                  2  Corp

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [27]:
# Step 1: Filter for customers who responded "Yes"
responders = df[df['response'] == 'Yes']

# Step 2: Group by policy_type and gender, and calculate relevant averages
summary = responders.groupby(['policy_type', 'gender']).agg({
    'monthly_premium_auto': 'mean',
    'customer_lifetime_value': 'mean',
    'total_claim_amount': 'mean'
}).reset_index()

# Step 3: Rename columns for clarity
summary.columns = [
    'Policy Type', 'Gender',
    'Avg Monthly Premium',
    'Avg Customer Lifetime Value',
    'Avg Total Claim Amount'
]

summary


,Policy Type,Gender,Avg Monthly Premium,Avg Customer Lifetime Value,Avg Total Claim Amount
0,Corporate Auto,F,94.301775,7712.628736,433.738499
1,Corporate Auto,M,92.188312,7944.465414,408.582459
2,Personal Auto,F,98.998148,8339.791842,452.965929
3,Personal Auto,M,91.085821,7448.383281,457.010178
4,Special Auto,F,92.314286,7691.584111,453.280164
5,Special Auto,M,86.343750,8247.088702,429.527942


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [17]:
# Step 1: Count customers per state
state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'customer_count']

# Step 2: Filter states with more than 500 customers
popular_states = state_counts[state_counts['customer_count'] > 500]
popular_states

,state,customer_count
0,California,3552
1,Oregon,2909
2,Arizona,1937
3,Nevada,993
4,Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [15]:
# Remove % and convert to float
df['customer_lifetime_value'] = df['customer_lifetime_value'].replace('%', '', regex=True).astype(float)
findings = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['min', 'max', 'median']).reset_index()
findings

,education,gender,min,max,median
0,Bachelor,F,1904.000852,73225.95652,5640.505303
1,Bachelor,M,1898.007675,67907.27050,5548.031892
2,College,F,1898.683686,61850.18803,5623.611187
3,College,M,1918.119700,61134.68307,6005.847375
4,Doctor,F,2395.570000,44856.11397,5332.462694
5,Doctor,M,2267.604038,32677.34284,5577.669457
6,High School or Below,F,2144.921535,55277.44589,6039.553187
7,High School or Below,M,1940.981221,83325.38119,6286.731006
8,Master,F,2417.777032,51016.06704,5729.855012
9,Master,M,2272.307310,50568.25912,5579.099207


## Bonus

In [ ]:
5. The marketing team wants to analyze the number of policies sold by state and month. 
Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [20]:
# Parse the date with the correct format to avoid the warning
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%Y-%m-%d', errors='coerce')

# Extract the month name
df['month'] = df['effective_to_date'].dt.month_name()

# Create the pivot table: rows = states, columns = months, values = policy count
policies_by_state_month = pd.pivot_table(
    df,
    index='state',
    columns='month',
    values='policy',  # Replace with a unique customer ID if more accurate
    aggfunc='count',
    fill_value=0
)

# Optional: Reorder columns to follow calendar months
month_order = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]
policies_by_state_month = policies_by_state_month.reindex(columns=month_order, fill_value=0)

# Display the result
print(policies_by_state_month)



month       January  February  March  April  May  June  July  August  \
state                                                                  
Arizona        1008       929      0      0    0     0     0       0   
California     1918      1634      0      0    0     0     0       0   
Nevada          551       442      0      0    0     0     0       0   
Oregon         1565      1344      0      0    0     0     0       0   
Washington      463       425      0      0    0     0     0       0   

month       September  October  November  December  
state                                               
Arizona             0        0         0         0  
California          0        0         0         0  
Nevada              0        0         0         0  
Oregon              0        0         0         0  
Washington          0        0         0         0  


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [ ]:
state_month_counts = df.groupby(['state', 'month']).size().reset_index(name='policies_sold')
top_states = total_by_state.sort_values(by='policies_sold', ascending=False).head(3)['state']
top_states_data = state_month_counts[state_month_counts['state'].isin(top_states)]
pivot_top_states = top_states_data.pivot_table(index='state', columns='month', values='policies_sold', aggfunc='sum', fill_value=0)

In [23]:
# 2. Extract the month name
df['month'] = df['effective_to_date'].dt.month_name()

# 3. Count number of policies sold by state and month
state_month_counts = df.groupby(['state', 'month']).size().reset_index(name='policies_sold')

# 4. Total policies sold by state
total_by_state = state_month_counts.groupby('state')['policies_sold'].sum().reset_index()

# 5. Get top 3 states by total policies sold
top_states = total_by_state.sort_values(by='policies_sold', ascending=False).head(3)['state']

# 6. Filter original grouped data to include only top 3 states
top_states_data = state_month_counts[state_month_counts['state'].isin(top_states)]

# 7. Pivot for a cleaner format (rows = states, columns = months)
pivot_top_states = top_states_data.pivot_table(
    index='state',
    columns='month',
    values='policies_sold',
    aggfunc='sum',
    fill_value=0
)
pivot_top_states = pivot_top_states.reindex(columns=month_order, fill_value=0)

# 9. Display result
print(pivot_top_states)

month       January  February  March  April  May  June  July  August  \
state                                                                  
Arizona        1008       929      0      0    0     0     0       0   
California     1918      1634      0      0    0     0     0       0   
Oregon         1565      1344      0      0    0     0     0       0   

month       September  October  November  December  
state                                               
Arizona             0        0         0         0  
California          0        0         0         0  
Oregon              0        0         0         0  


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here